<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Cost Of Care Analytics Demo
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction:</b></p>
<p style = 'font-size:16px;font-family:Arial'>With rising healthcare expenditures, there's a growing urgency to track, analyze, and optimize cost drivers across emergency room (ER), imaging, surgical, and mental health services. Traditional cost auditing methods are retrospective, siloed, and lack the granularity to enable proactive cost containment. By leveraging advanced analytics and integrated hospital datasets, a scalable, dynamic, and contextual cost intelligence solution can be achieved. However, key challenges remain: 
</p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Fragmented cost data across procedures, diagnoses, and provider systems.</li>
    <li>Lack of context linking ER visits, imaging, surgeries, and mental health costs to patient pathways.</li>
    <li>Limited visual insights into cost trends across time, region, and condition severity.</li>
    <li>Need for scalable, in-database analytics to handle massive healthcare datasets.</li>
    
</ul>
<p style = 'font-size:18px;font-family:Arial'><b>Proposed Solution:</b></p>
<p style = 'font-size:16px;font-family:Arial'>The proposed solution leverages Teradata Vantage as an intelligent analytics platform to consolidate, process, and visualize healthcare cost data across clinical domains. Structured datasets from emergency room (ER) visits, imaging studies, surgical procedures, and mental health conditions were ingested into the Vantage environment </p>

<ul style = 'font-size:16px;font-family:Arial'>
    <li><b>Annual-Across Dashboard:</b><br>
    Tracks the total and proportionate number of ER visits year-over-year, highlighting avoidable and treatable cases. This allows stakeholders to identify cost reduction opportunities by focusing on potentially preventable ER encounters.</li>
    <li><b>Mental Health Dashboard:</b><br>
    Displays the yearly trend of mental health–related diagnoses, illustrating a sharp increase in recent years. This trend emphasizes the growing importance of behavioral health management and targeted interventions to contain rising costs.</li>
    <li><b>Top Diagnoses Dashboard:</b><br>
    Analyzes the top emergency visit drivers and their associated per-1,000-patient rates and Net Paid PMPM (Per Member Per Month) costs. Conditions such as cancer, residual complications, and viral infections emerge as leading cost contributors—helping prioritize care management programs.</li>
    <li><b>X-Ray / Echo Tracker Dashboard:</b><br>
    Provides patient- and provider-level utilization analytics for imaging tests like X-rays and echocardiograms. These insights identify overutilization patterns and support efficiency initiatives in radiology departments.</li>
    <li><b>Imaging Studies Dashboard:</b><br>
    Aggregates annual and monthly imaging volumes and costs. Trends reveal a substantial growth in imaging utilization and related expenditures, driving the need for predictive capacity planning and budget alignment.</li>
</ul>

<p style = 'font-size:16px;font-family:Arial'>Using these curated datasets, interactive dashboards were developed in Superset to provide a unified “Cost of Care” view that enables value-based decision-making:
</p>
<p style = 'font-size:18px;font-family:Arial'><b>Benefits:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>End-to-end cost visibility across all major healthcare services—ER, imaging, surgery, and mental health.</li>
    <li>Identification of high-cost hospitals and procedures with spatial and temporal granularity.</li>
    <li>Scalable feature engineering and analytics via Teradata’s in-database capabilities.</li>
    <li>Supports hospital value-based purchasing (HBVP) and patient-centric care strategies.</li>
    <li>Interactive and user-friendly dashboards enable deep dives into cost outliers and overutilization patterns.</li>
</ul>


<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial;'><b>1. Connect to Vantage, Import python packages and explore the dataset</b></p>

In [ ]:
#import libraries
import getpass
from teradataml import *

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

display.max_rows=5

<hr style="height:2px;border:none;">
<b style = 'font-size:18px;font-family:Arial;'> 1.1 Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=FF_CostOfCare_Healthcare.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:18px;font-family:Arial'> <b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_HealthcareCOC_cloud');"
# takes about 3 minutes
#%run -i ../run_procedure.py "call get_data('DEMO_HealthcareCOC_local');" 
# takes about 15minutes

<p style = 'font-size:16px;font-family:Arial'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>2. Data Exploration </b></p>

In [ ]:
db_list_tables('DEMO_HealthcareCOC')

<p style = 'font-size:18px;font-family:Arial'><b>Data Sources and Schema Overview:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li><b>Va_Encounters:</b>
    Contains detailed records of patient encounters, including provider, payer, costs, and visit type.</li>
    <li><b>Va_Medications:</b>
    Tracks prescribed medications with cost, payer coverage, and associated encounter information.</li>
    <li><b>Va_Procedures:</b>
    Lists performed medical procedures with codes, descriptions, and base cost data.</li>
    <li><b>Va_Conditions:</b>
    Records diagnosed patient conditions with start/stop dates, codes, and encounter references.</li>
    <li><b>Va_Immunizations:</b>
    Logs administered immunizations with associated patient, encounter, and cost details.</li>
    <li><b>Va_Providers:</b>
    Holds provider demographic, specialty, and utilization information for encounter linkage.</li>
    <li><b>va_payers:</b>
    Stores payer or insurance organization details, coverage amounts, and financial performance metrics.</li>
    <li><b>va_bk_diag_nyu:</b>
    Diagnostic mapping table classifying conditions by avoidability and emergency department category.</li>
    <li><b>MI_primary_condition_mapping:</b>
    Maps primary medical condition descriptions to standardized clinical categories for analysis.</li>
    <li><b>MI_secondary_condition_mapping:</b>
   Provides secondary condition-to-category mappings used for comorbidity and risk grouping.</li>
    <li><b>MI_hospitalscores:</b>
    Contains hospital performance and quality scores across clinical, safety, and efficiency domains.</li>

<p style = 'font-size:16px;font-family:Arial'>As we can see from above, we have data in tables about the Procedure, conditions, medications etc. </p>

In [ ]:
df_proc = DataFrame(in_schema("DEMO_HealthcareCOC", "Va_Procedures"))
df_proc

In [ ]:
df_proc = DataFrame(in_schema("DEMO_HealthcareCOC", "Va_Conditions"))
df_proc

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>3. Dashboard & Insights </b></p>

<p style='font-size:16px;font-family:Arial;color:#00233C'> This section highlights an interactive overview of our Cost of Care Analytics. The dashboard visualizes key metrics related to emergency room utilization, imaging and surgical procedures, and cost drivers across conditions and hospitals. It provides a holistic understanding of how healthcare costs are distributed and where intervention opportunities lie. </p> <p style='font-size:16px;font-family:Arial;color:#00233C'> Key features of the dashboard: <ul style='font-size:16px;font-family:Arial;color:#00233C'> <li>Bar charts showing annual trends in ER visits and costs across multiple categories like Avoidable, Treatable, and Not Preventable conditions.</li> <li>Stacked visualizations depicting proportionate cost contribution by diagnosis and procedure type.</li> <li>Interactive selectors to filter top diagnoses and providers by year, enabling targeted drill-downs.</li> <li>Imaging and surgical trackers to monitor frequency and cost over time for X-rays, MRIs, CT scans, and high-cost surgeries.</li> <li>Geo-mapped insights on hospital performance scores and average procedure costs across counties.</li> </ul> </p>

<p style='font-size:16px;font-family:Arial;color:#00233C'><b>Dashboard Snapshot:</b></p>
<img src="images/cost_of_care.jpg" style="border: 4px solid #404040; border-radius: 10px;"/>

<p style = 'font-size:16px;font-family:Arial'>
<div style="text-align: left; ">
        <a href="https://superset.env.ci.clearscape.teradata.com/superset/dashboard/p/Yv7OP59LyNp/" style="padding: 10px; background-color: #00233c; color: #FFFFFF; text-decoration: none; border-radius: 5px;">
         Cost Of Care Dashboard


<p style='font-size:16px;font-family:Arial;color:#00233C'>
The dashboard is structured across multiple tabs to allow detailed investigation:
</p>

<ul style='font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Annual-Accross:</b> Tracks ER visits and preventability over time.</li>
    <li><b>Top Diagnoses:</b> Ranks primary and secondary conditions based on cost and frequency.</li>
    <li><b>Xray / Echo Tracker:</b> Flags patients and providers with excessive imaging.</li>
    <li><b>Imaging Studies:</b> Evaluates yearly and monthly volumes and costs of major scans.</li>
    <li><b>Surgical Procedures:</b> Tracks trends and maps surgery cost by region.</li>
    <li><b>Mental Health:</b> Monitors prevalence of selected mental disorders over time.</li>
    <li><b>County Health Rankings:</b> Geospatial overlay of public health indicators across counties.</li>
</ul>

<p style='font-size:16px;font-family:Arial;color:#00233C'>
Each tab enables interactive filtering by year, condition, category, and provider to customize the view and drive targeted insights.
</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_HealthcareCOC');"        # Takes 10 seconds

In [ ]:
remove_context()

<footer style="padding-bottom: 35px; background: #f9f9f9; border-bottom: 3px solid #00233C">
    <div style="float: left; margin-top: 14px;">ClearScape Analytics™</div>
    <div style="float: right;">
        <div style="float: left; margin-top: 14px;">Copyright © Teradata Corporation - 2025. All Rights Reserved.</div>
    </div>
</footer>